
# Gaussian Naive Bayes στο Iris dataset

Σε αυτό το notebook θα δούμε ένα απλό παράδειγμα χρήσης
**Gaussian Naive Bayes** στο κλασικό **Iris dataset**.

## Στόχοι

- Να θυμηθούμε τη βασική ιδέα του Naive Bayes.
- Να δούμε πότε χρησιμοποιούμε την παραλλαγή **Gaussian**.
- Να εκπαιδεύσουμε GaussianNB στο Iris dataset.
- Να αξιολογήσουμε την απόδοση με confusion matrix.
- Να δούμε posterior πιθανότητες για μερικά δείγματα.


## Θεωρία: Gaussian Naive Bayes

Σε αυτό το τμήμα περιγράφουμε τη θεωρία πίσω από τον ταξινομητή **Gaussian Naive Bayes** (Γκαουσιανό Naive Bayes classifier), ξεκινώντας από τον γενικό κανόνα του Naive Bayes, περνώντας στο Γκαουσιανό μοντέλο με συναρτήσεις πυκνότητας πιθανότητας (probability density functions, **pdf**) και φτάνοντας στη λογαριθμική κλίμακα (log-scale), τον υπολογισμό των εκ των υστέρων πιθανοτήτων (posterior probabilities) και τη χρήση τυποποίησης (standardization) με z-scores.

---

### 1. Γενικός κανόνας Naive Bayes

Θεωρούμε ένα διάνυσμα χαρακτηριστικών (feature vector) 
$$
x = (x_1, x_2, \ldots, x_d)
$$
και μια διακριτή μεταβλητή κλάσης (class label) $y$ που παίρνει τιμές σε ένα σύνολο κλάσεων $\{1, \dots, K\}$.

Ο βασικός κανόνας του **Naive Bayes** βασίζεται στον κανόνα του Bayes και στην υπόθεση συνθήκης ανεξαρτησίας (conditional independence assumption) των χαρακτηριστικών, δεδομένης της κλάσης, και μας δίνει την **εκ των υστέρων πιθανότητα** (posterior probability) $P(y \mid x)$ ως:

$$
P(y \mid x) \propto P(y) P(x \mid y)
= P(y) \prod_{i=1}^d P(x_i \mid y).
$$

(πλήρης κανονικοποιημένος τύπος Bayes:)

$$
P(y \mid x)
= \frac{P(y)\,P(x \mid y)}{\sum_{k} P(y=k)\,P(x \mid y=k)}.
$$

- $P(y)$: εκ των προτέρων πιθανότητα (prior probability) για την κλάση $y$.
- $P(x \mid y)$: πιθανοφάνεια (likelihood) των δεδομένων $x$ δεδομένης της κλάσης $y$.
- $P(y \mid x)$: εκ των υστέρων πιθανότητα (posterior probability) της κλάσης $y$ δεδομένων των χαρακτηριστικών $x$.
- $P(x_i \mid y)$: επιμέρους όροι πιθανοφάνειας (likelihood terms) για κάθε χαρακτηριστικό (feature).

Η υπόθεση Naive (Naive assumption) είναι ότι, δεδομένης της κλάσης $y$, τα χαρακτηριστικά $x_i$ είναι ανεξάρτητα μεταξύ τους· γι’ αυτό η πιθανοφάνεια γράφεται ως γινόμενο.

---

### 2. Γκαουσιανό Naive Bayes και συνάρτηση πυκνότητας πιθανότητας (probability density function, pdf)

Στο **Gaussian Naive Bayes** υποθέτουμε ότι κάθε χαρακτηριστικό (feature) $x_i$, για κάθε κλάση $y = k$, ακολουθεί Κανονική κατανομή (Gaussian distribution). Δηλαδή:

$$
P(x_i \mid y = k) 
= \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2)
= \frac{1}{\sqrt{2\pi\sigma_{k,i}^2}}
  \exp\left(-\frac{(x_i-\mu_{k,i})^2}{2\sigma_{k,i}^2}\right),
$$

όπου:

- $\mu_{k,i}$: μέση τιμή (mean) του χαρακτηριστικού $i$ στην κλάση $k$,
- $\sigma_{k,i}^2$: διακύμανση (variance) του χαρακτηριστικού $i$ στην κλάση $k$.

Αυτή η συνάρτηση είναι η **συνάρτηση πυκνότητας πιθανότητας** (probability density function, **pdf**) της Κανονικής κατανομής.

Για ένα πλήρες διάνυσμα χαρακτηριστικών $x$, η συνολική πιθανοφάνεια (likelihood) υπό την κλάση $y = k$ γράφεται ως γινόμενο των Γκαουσιανών:

$$
P(x \mid y = k) = \prod_{i=1}^d \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2).
$$

Επομένως η εκ των υστέρων πιθανότητα (posterior probability) γράφεται:

$$
P(y = k \mid x) \propto P(y = k)
\prod_{i=1}^d \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2).
$$

Το Gaussian Naive Bayes είναι ιδιαίτερα κατάλληλο όταν τα χαρακτηριστικά είναι
**συνεχείς αριθμητικές μεταβλητές** (continuous numeric features), π.χ. μήκη, βάρη, φυσικές μετρήσεις κ.λπ.

Αν για ένα συγκεκριμένο χαρακτηριστικό (π.χ. μήκος πετάλου (petal length)) οι μέσοι και οι διακυμάνσεις διαφέρουν έντονα ανά κλάση, τότε η αντίστοιχη Γκαουσιανή pdf συμβάλλει ισχυρή διακριτική πληροφορία στο $P(x \mid y = k)$, αυξάνοντας τη διαχωρισιμότητα (separability) των κλάσεων.

---

### 3. Μάθηση παραμέτρων από δεδομένα (empirical parameter estimation)

Κατά την εκπαίδευση του μοντέλου Gaussian Naive Bayes, οι παράμετροι εκτιμώνται **εμπειρικά** (empirically) από τα δεδομένα εκπαίδευσης (training data).

Στη scikit-learn, με την κλήση:

```python
gnb.fit(X_train_scaled, y_train)
```

το αντικείμενο `GaussianNB` εκτιμά:

- $N_k$ = πλήθος δειγμάτων στην κλάση (class) $k$  
  → `gnb.class_count_`.

- $\hat{\pi}_k = \dfrac{N_k}{N}$ = εμπειρική εκ των προτέρων πιθανότητα (empirical prior probability) για την κλάση $k$  
  → `gnb.class_prior_`.

- $\hat{\mu}_{k,i} = \dfrac{1}{N_k} \sum_{n: y_n=k} x_{n,i}$  
  μέσος (mean) του χαρακτηριστικού $i$ στην κλάση $k$  
  → `gnb.theta_`.

- $\hat{\sigma}^2_{k,i} = \dfrac{1}{N_k} \sum_{n: y_n=k} (x_{n,i} - \hat{\mu}_{k,i})^2$  
  διακύμανση (variance) του χαρακτηριστικού $i$ στην κλάση $k$  
  → `gnb.var_`.

Αυτές οι εκτιμήσεις είναι τυπικά **μέγιστης πιθανοφάνειας** (maximum likelihood estimates, MLE) υπό την υπόθεση της Γκαουσιανής κατανομής.

---

### 4. Συνάρτηση πυκνότητας πιθανότητας (pdf) σε λογαριθμική κλίμακα (log-scale)

Στην πράξη, το Gaussian Naive Bayes δεν δουλεύει απευθείας με τις τιμές της pdf 
$P(x_i \mid y=k)$, αλλά με τον **λογάριθμο της pdf** (log-pdf) για κάθε χαρακτηριστικό.  

Για την Κανονική κατανομή έχουμε:

$$
\log P(x_i \mid y=k)
= \log \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2)
= -\frac{1}{2}
\left[
\log(2\pi\sigma_{k,i}^2)
+
\frac{(x_i - \mu_{k,i})^2}{\sigma_{k,i}^2}
\right].
$$

Παρατηρούμε ότι:

- Ο πολλαπλασιαστικός παράγοντας 
  $\dfrac{1}{\sqrt{2\pi\sigma_{k,i}^2}}$
  γίνεται πρόσθετος όρος 
  $-\frac{1}{2}\log(2\pi\sigma_{k,i}^2)$
  στο log-scale.
- Ο εκθετικός όρος 
  $\exp\left(-\frac{(x_i-\mu_{k,i})^2}{2\sigma_{k,i}^2}\right)$
  μετατρέπεται σε γραμμικό όρο 
  $-\dfrac{(x_i-\mu_{k,i})^2}{2\sigma_{k,i}^2}$
  μέσα στα brackets.

Για ένα πλήρες διάνυσμα χαρακτηριστικών $x$, η log-πιθανοφάνεια (log-likelihood) για την κλάση $k$ είναι:

$$
\log P(x \mid y = k)
= \sum_{i=1}^d \log P(x_i \mid y = k).
$$

Αυτό αντιστοιχεί στον κώδικα μιας συνάρτησης τύπου `log_gaussian_pdf(x, mean, var)` που υπολογίζει τον log-pdf για κάθε χαρακτηριστικό και στη συνέχεια τα αθροίζει.

---

### 5. Λογαριθμική κλίμακα (log-scale), log-likelihood και αριθμητική σταθερότητα (numerical stability)

Αντί να δουλεύουμε με τα γινόμενα πιθανοτήτων στο αρχικό τους μέγεθος, περνάμε σε **λογαριθμική κλίμακα (log-scale)**:

- Αν
  $$
  P(x \mid y=k) = \prod_{i=1}^d p_{k,i},
  $$
  τότε
  $$
  \log P(x \mid y=k) = \sum_{i=1}^d \log p_{k,i}.
  $$

Αυτό έχει τρία βασικά πλεονεκτήματα:

1. **Αριθμητική σταθερότητα (numerical stability)**  
   Οι πιθανότητες είναι αριθμοί στο $(0,1)$ και όταν πολλαπλασιάζουμε πολλά τέτοια νούμερα, το γινόμενο μπορεί να γίνει τόσο μικρό που να «υποχειλίσει» (underflow) σε υπολογιστή και να μη διακρίνεται από το 0.  
   Με τους λογαρίθμους:
   - μετατρέπουμε το γινόμενο σε άθροισμα,
   - δουλεύουμε με (συνήθως) μέτριες αρνητικές τιμές (π.χ. -5, -10, -100 αντί για $10^{-23}$),
   - αποφεύγουμε αριθμητικά σφάλματα υποχειλισμού.

2. **Απλούστεροι υπολογισμοί (simpler computations)**  
   Στο Naive Bayes η πιθανοφάνεια είναι γινόμενο πιθανοτήτων. Σε log-κλίμακα:
   - οι log-likelihoods απλά αθροίζονται,
   - μπορούμε να προσθέσουμε εύκολα log-priors και log-likelihoods για να πάρουμε log-posteriors.

3. **Αναλλοίωτο του argmax (argmax invariance)**  
   Για την απόφαση κλάσης χρησιμοποιούμε τον εκτιμητή μέγιστης εκ των υστέρων πιθανότητας (maximum a posteriori estimator, **MAP**):

   $$
   \hat y(x) = \arg\max_k P(y=k \mid x).
   $$

   Επειδή ο λογάριθμος είναι **αυστηρά αύξουσα** συνάρτηση, ισχύει:

   $$
   \arg\max_k P(y=k \mid x)
   = \arg\max_k \log P(y=k \mid x).
   $$

   Άρα μπορούμε να δουλεύουμε αποκλειστικά με log-τιμές χωρίς να αλλάζει η τελική απόφαση του ταξινομητή.

---

### 6. Log-posterior στο Gaussian Naive Bayes και MAP estimator

Θυμόμαστε ότι:

$$
P(y=k \mid x) \propto P(y=k)\,P(x \mid y=k).
$$

Σε λογαριθμική μορφή:

$$
\log P(y=k \mid x)
= \log P(y=k) + \log P(x \mid y=k) + \text{σταθερά},
$$

όπου η «σταθερά» δεν εξαρτάται από το $k$ και άρα μπορεί να αγνοηθεί στον υπολογισμό του $\arg\max$.

Στο Gaussian Naive Bayes αυτό γίνεται:

$$
\log P(y=k \mid x)
\propto
\log \pi_k
+
\sum_{i=1}^d \log \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2),
$$

οπότε ο πρακτικός υπολογισμός της log-posterior για την κλάση $k$ είναι:

$$
\text{log\_post}_k
= \log \pi_k
+ \sum_{i=1}^d \log \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2).
$$

Ο εκτιμητής MAP (maximum a posteriori estimator) επιλέγει την κλάση:

$$
\hat y(x)
= \arg\max_k \text{log\_post}_k.
$$

Αυτό είναι ακριβώς αυτό που υλοποιεί η μέθοδος `predict(x)` στο `GaussianNB`.

---

### 7. Από log-posterior σε κανονικές πιθανότητες (log-sum-exp trick και softmax)

Για να πάρουμε **κανονικοποιημένες εκ των υστέρων πιθανότητες** (normalized posterior probabilities), ώστε να αθροίζονται στο 1, πρέπει να μετατρέψουμε τις log-τιμές σε «κανονικές» πιθανότητες.

Αν έχουμε ένα διάνυσμα log-τιμών:

$$
z_k = \text{log\_post}_k,
$$

τότε ο απλός ορισμός θα ήταν:

$$
p_k = \frac{e^{z_k}}{\sum_j e^{z_j}}.
$$

Ωστόσο, για να αποφύγουμε αριθμητικά προβλήματα (overflow/underflow), χρησιμοποιούμε ένα κόλπο αριθμητικής σταθερότητας, γνωστό ως **log-sum-exp trick**:

1. Υπολογίζουμε τη μέγιστη log-τιμή:

   $$
   m = \max_k z_k.
   $$

2. «Μετατοπίζουμε» (shift) όλα τα $z_k$:

   $$
   z'_k = z_k - m.
   $$

3. Υπολογίζουμε:

   $$
   p_k = \frac{e^{z'_k}}{\sum_j e^{z'_j}}.
   $$

Επειδή όλες οι πιθανότητες έχουν πολλαπλασιαστεί με τον ίδιο παράγοντα $e^{-m}$, η κανονικοποίηση διορθώνει αυτόν τον παράγοντα και καταλήγουμε σε σωστές πιθανότητες χωρίς αριθμητικά προβλήματα.

Αυτός ο μετασχηματισμός είναι ισοδύναμος με τη συνάρτηση **softmax**:

$$
\operatorname{softmax}(z)_k
=
\frac{e^{z_k}}{\sum_j e^{z_j}},
$$

απλώς υλοποιημένη με τρόπο ανθεκτικό σε αριθμητικά σφάλματα (numerically stable implementation).

---

### 8. Βάση λογαρίθμου και ερμηνεία

Στο `GaussianNB` (και γενικά στη scikit-learn) χρησιμοποιείται ο **φυσικός λογάριθμος** (natural logarithm, βάση $e$).  
Αν αντί για $\ln$ χρησιμοποιούσαμε $\log_{10}$, η διαφορά θα ήταν μόνο ένας σταθερός παράγοντας:

$$
\log_{10} x = \frac{1}{\ln 10} \ln x.
$$

Επειδή ο λογάριθμος είναι μονοτονικά αύξουσα συνάρτηση (monotonically increasing function), ο παράγοντας αυτός:

- δεν επηρεάζει το ποια κλάση $k$ μεγιστοποιεί το $\log P(y=k \mid x)$,
- δεν επηρεάζει τις κανονικοποιημένες posterior πιθανότητες μετά την εφαρμογή της softmax.

Άρα για τον ταξινομητή η επιλογή βάσης του λογαρίθμου **δεν αλλάζει το αποτέλεσμα**, απλώς αλλάζει την κλίμακα (scale) των log-τιμών.

---

### 9. Τυποποίηση (standardization) και z-scores

Συχνά, πριν από την εκπαίδευση του Gaussian Naive Bayes, εφαρμόζουμε **τυποποίηση** (standardization) σε κάθε χαρακτηριστικό (feature) ώστε όλα τα μεγέθη να βρίσκονται στην ίδια κλίμακα.

Για κάθε χαρακτηριστικό $x_i$ ορίζουμε το αντίστοιχο z-score:

$$
z_i = \frac{x_i - \mu_i}{\sigma_i},
\quad
\mu_i = \frac{1}{n}\sum_{j=1}^n x_{j,i},
\quad
\sigma_i = \sqrt{\frac{1}{n}\sum_{j=1}^n (x_{j,i}-\mu_i)^2}.
$$

Οι τυποποιημένες τιμές (standardized values, **z-scores**):

- έχουν δειγματικό μέσο περίπου $E[z_i] \approx 0$,
- και διακύμανση περίπου $\operatorname{Var}(z_i) \approx 1$.

Σε αυτό το νέο σύστημα συντεταγμένων (z-scale για τα features, log-scale για τις πιθανότητες):

- το GaussianNB εκτιμά τις παραμέτρους $\hat{\mu}_{k,i}$ και $\hat{\sigma}^2_{k,i}$ πάνω σε **z-scores**,
- η log-likelihood γράφεται στη λογαριθμική κλίμακα (log-scale),
- και οι αποφάσεις παίρνονται με βάση τον MAP estimator πάνω στις log-posteriors.

Πολύ σημαντικό για να αποφύγουμε **διαρροή πληροφορίας** (data leakage):

- τα $\hat{\mu}_i$ και $\hat{\sigma}_i$ του StandardScaler υπολογίζονται **μόνο** στο σύνολο εκπαίδευσης (training set),
- ο ίδιος μετασχηματισμός εφαρμόζεται στη συνέχεια στα σύνολα επικύρωσης και ελέγχου (validation/test sets).

Έτσι:

- κάθε χαρακτηριστικό συμβάλλει ισότιμα στην πιθανοφάνεια (likelihood),
- οι Γκαουσιανές (Gaussians) δεν αλλοιώνονται από ετερογενείς μονάδες μέτρησης,
- και η χρήση της λογαριθμικής κλίμακας (log-scale) εξασφαλίζει αριθμητική σταθερότητα και καθαρή θεωρητική ερμηνεία των εκ των υστέρων πιθανοτήτων (posterior probabilities).


## Το Iris dataset και τα χαρακτηριστικά του (Iris dataset & features)

Το Iris dataset αποτελεί ένα κλασικό σύνολο δεδομένων για επίδειξη τεχνικών ταξινόμησης. Περιλαμβάνει μετρήσεις από άνθη τριών ειδών Iris και κάθε εγγραφή (γραμμή) του dataset αντιστοιχεί σε ένα μεμονωμένο δείγμα άνθους (sample / observation).

Συνοπτική δομή:

- Γραμμές (rows / samples): κάθε γραμμή είναι ένα άνθος (π.χ. ένα δείγμα), και στο Iris dataset έχουμε συνολικά 150 δείγματα.
- Στήλες (columns / features): περιλαμβάνουν τις αριθμητικές μετρήσεις που χρησιμοποιούνται ως είσοδοι (features) στον ταξινομητή, καθώς και την στήλη-ετικέτα (label / target):
  - `sepal length (cm)` — μήκος του sepal σε εκατοστά (centimeters).
  - `sepal width (cm)` — πλάτος του sepal σε εκατοστά.
  - `petal length (cm)` — μήκος του petal σε εκατοστά.
  - `petal width (cm)` — πλάτος του petal σε εκατοστά.
  - `species` (target / label) — είδος του άνθους: `setosa`, `versicolor`, `virginica`.

Βιολογική/φυσική σημασία των χαρακτηριστικών:

- Το **sepal** (σέπαλο) είναι η εξωτερική δομή που προστατεύει το μπουμπούκι (flower bud). Το **petal** (πέταλο) είναι συνήθως χρωματιστή δομή που προσελκύει επικονιαστές (insects / pollinators) και σχετίζεται με αναπαραγωγικούς μηχανισμούς.

- Τα **petal features** (μήκος/πλάτος πέταλου) συχνά φέρουν περισσότερη πληροφορία για τη διαφορετικότητα των ειδών λόγω διαφοροποίησης που σχετίζεται με επικονιαστές και λειτουργία άνθισης. Γι' αυτό στα δεδομένα του Iris συνήθως αποτελούν τα πιο διαχωριστικά χαρακτηριστικά.

- Τα **sepal features** (μήκος/πλάτος σέπαλου) εμφανίζουν συχνά μικρότερη διασπορά ανάμεσα στα είδη και μπορεί να είναι λιγότερο διακριτικά.

Συμβουλές κατά την ανάλυση:

- Επιβεβαίωσε ότι οι μονάδες (centimeters) έχουν ενιαία μορφή και ελέγξτε για missing values πριν την ανάλυση (π.χ. `df.isnull().sum()`).
- Προτού εκπαιδεύσετε μοντέλα, συχνά εφαρμόζουμε **τυποποίηση (StandardScaler)** ώστε τα χαρακτηριστικά σε διαφορετικές κλίμακες να γίνουν συγκρίσιμα (μηδενικός μέσος, μονάδα τυπικής απόκλισης).

Σύνδεση με ταξινόμηση και GaussNB:

- Τα χαρακτηριστικά χρησιμοποιούνται ως `X` (features), ενώ η στήλη `species` ως `y` (target/label).
- Το Gaussian Naive Bayes (GaussianNB) χρησιμοποιεί στατιστικές παραμέτρους (μέσους / διακυμάνσεις ανά κλάση) για να μοντελοποιήσει τις πιθανότητες P(x | y) και να υπολογίσει την εκ των υστέρων πιθανότητα (posterior) P(y | x).

In [ ]:
# Βασικά imports: εισάγουμε τις βιβλιοθήκες που θα χρησιμοποιήσουμε
# numpy: αριθμητικές πράξεις και πίνακες
import numpy as np
# matplotlib: plotting/γραφικές παραστάσεις
import matplotlib
# Force inline backend to avoid VSCode Plot Viewer duplicates and ensure inline rendering
matplotlib.use('module://matplotlib_inline.backend_inline')
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap, BoundaryNorm

# scikit-learn: φορτώνουμε το iris dataset, κάνουμε split, scaling και GaussianNB
from sklearn.datasets import load_iris  # φορτώνει το κλασικό Iris dataset
from sklearn.model_selection import train_test_split  # χωρισμός train/validation
from sklearn.preprocessing import StandardScaler  # κανονικοποίηση/standardization
from sklearn.naive_bayes import GaussianNB  # Gaussian Naive Bayes classifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay  # μετρικές αξιολόγησης


In [ ]:
# Φόρτωση του Iris dataset

iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
class_names = iris.target_names

# Recompute canonical colormap now that we know class_names length
cmap = plt.get_cmap('tab10')
# Create discrete colormap and normalization for class integers (0..K-1)
K = len(class_names)
class_colors = [cmap(i) for i in range(K)]
class_cmap = ListedColormap(class_colors)
class_norm = BoundaryNorm(np.arange(K + 1) - 0.5, K)
# Also keep CLASS_COLORS as hex list for textual usage and CLASS_COLOR_MAP mapping
CLASS_COLORS = [mcolors.to_hex(c) for c in class_colors]
CLASS_COLOR_MAP = dict(zip(class_names, CLASS_COLORS))
# Optionally disable static scatter plots if you prefer a single interactive figure (set to True/False)

print("Σχήμα X:", X.shape)
print("Χαρακτηριστικά:", feature_names)
print("Κλάσεις:", class_names)
print('CLASS_COLORS =', CLASS_COLORS)


In [ ]:

# Ρίχνουμε μια πρώτη ματιά στα δεδομένα (πρώτα 5 δείγματα)

X[:5], y[:5]


In [ ]:
# Απλή 2D απεικόνιση: petal length vs petal width
# Δημιουργούμε ένα figure και ένα axis για να σχεδιάσουμε
fig, ax = plt.subplots()

# Σχεδιάζουμε scatter plot: X[:, 2] = petal length, X[:, 3] = petal width
# Use discrete class colormap (class_cmap) with class_norm to map integer classes consistently
scatter = ax.scatter(X[:, 2], X[:, 3], c=y, cmap=class_cmap, norm=class_norm, alpha=0.8)

# Δημιουργία απλού legend (Line2D handles with CLASS_COLORS)
from matplotlib.lines import Line2D
legend_colors = CLASS_COLORS
handles = [Line2D([0], [0], marker='o', color=legend_colors[i], linestyle='', markersize=8) for i in range(len(class_names))]
ax.legend(handles, class_names, title='Class')  # π.χ. setosa, versicolor, virginica

# Βάζουμε ετικέτες στους άξονες και τίτλο
ax.set_xlabel("petal length (cm)")  # μήκος πετάλου
ax.set_ylabel("petal width (cm)")   # πλάτος πετάλου
ax.set_title("Iris – petal length vs petal width")

# Κάνουμε tight layout για καλύτερη εμφάνιση
fig.tight_layout()
plt.show()


## Ερμηνεία scatter plot (petal length vs petal width)

- Άξονες:
  - Οριζόντιος άξονας: `petal length (cm)` — μήκος πετάλου σε εκατοστά.
  - Κατακόρυφος άξονας: `petal width (cm)` — πλάτος πετάλου σε εκατοστά.

- Κωδικοποίηση δεδομένων:
  - Κάθε σημείο αντιπροσωπεύει ένα δείγμα (παρατήρηση / sample) άνθους.
  - Το χρώμα δείχνει την πραγματική κλάση (species / target): `setosa`, `versicolor`, `virginica`.
  - Το legend (υπόμνημα) στο διάγραμμα δείχνει ποιο χρώμα αντιστοιχεί σε ποια κλάση.

Σκοπός του διαγράμματος:
- Ελέγχουμε τη **διαχωρισιμότητα (separability)** των κλάσεων στον χώρο των χαρακτηριστικών που έχει μεγαλύτερη πληροφορία: τα χαρακτηριστικά `petal length/width` συχνά παρέχουν ισχυρή διάκριση μεταξύ των ειδών.

Τι παρατηρούμε :
1. Η κλάση `Iris setosa` εμφανίζεται ως ξεκάθαρο, απομονωμένο cluster  με μικρές τιμές petal length/width — εξηγεί γιατί ο ταξινομητής προβλέπει setosa με πολύ υψηλή ακρίβεια (precision & recall).
2. Οι κλάσεις `Iris versicolor` και `Iris virginica` παρουσιάζουν μερική επικάλυψη (overlap) στα petal features — γι' αυτό παρουσιάζονται περισσότερες λάθος εκτιμήσεις (misclassifications) ανάμεσά τους.

Σύνδεση με μοντέλο (model connection):
- Το διάγραμμα δείχνει γιατί ένα απλό μοντέλο όπως το GaussianNB μπορεί να διαχωρίσει εύκολα την `setosa` αλλά να κάνει λάθη ανάμεσα σε `versicolor` και `virginica` στις περιοχές όπου οι κατανομές επικαλύπτονται.
- Στα σημεία επικάλυψης η **εκ των υστέρων πιθανότητα (posterior probability)** είναι πιο ισορροπημένη (ελάχιστα διακριτή) και το `predict_proba()` θα δείξει ανάλογα μικρή βεβαιότητα.

Σημεία σχολιασμού :
- Οι άξονες προβάλλονται σε φυσικές μονάδες (cm), αλλά πριν την εκπαίδευση χρησιμοποιούμε συχνά **τυποποίηση (standardization)** ώστε να συγκρίνονται τα χαρακτηριστικά.
- Το διάγραμμα είναι χρήσιμο για επιλογή χαρακτηριστικών (feature selection) και για την ερμηνεία των σφαλμάτων που παρατηρούμε στο confusion matrix.


In [ ]:
# Εξερεύνηση: Boxplots για κάθε χαρακτηριστικό ανά κλάση
# Χρησιμοποιούμε pandas για εύκολο grouping και plotting
import pandas as pd

# Δημιουργία DataFrame με ονόματα χαρακτηριστικών
iris_df = pd.DataFrame(X, columns=feature_names)
iris_df['species'] = [class_names[idx] for idx in y]

# Εξερεύνηση: Boxplots για κάθε χαρακτηριστικό ανά κλάση — απλή και φιλική για αρχάριους
# Χρησιμοποιούμε Matplotlib.boxplot with colors derived from tab10 colormap
fig, axes = plt.subplots(1, 4, figsize=(18, 4))
for i, feat in enumerate(feature_names):
    # Δεδομένα: μια λίστα με τις τιμές κάθε κλάσης για το χαρακτηριστικό feat
    data_per_class = [iris_df[iris_df['species'] == cls][feat].values for cls in class_names]
    # Σχεδιάζουμε grouped boxplot — κάθε στοιχείο του data_per_class είναι μια ομάδα
    bplot = axes[i].boxplot(data_per_class, tick_labels=class_names, patch_artist=True)
    # Ορίζουμε τα χρώματα των κουτιών (boxes) με βάση το class_colors
    for patch, color in zip(bplot['boxes'], class_colors):
        patch.set_facecolor(color)
        patch.set_edgecolor('k')
    axes[i].set_title(feat)
    axes[i].set_xlabel('')
fig.tight_layout()
plt.show()


In [ ]:
# Εξερεύνηση: pairwise scatterplots για μερικούς συνδυασμούς χαρακτηριστικών
# Επιλέγουμε μερικά ζεύγη χαρακτηριστικών για να ελέγξουμε συμμετρία/συσχέτιση
pairs = [
    ('sepal length (cm)', 'sepal width (cm)'),
    ('petal length (cm)', 'petal width (cm)'),
    ('sepal length (cm)', 'petal length (cm)'),
    ('sepal width (cm)', 'petal width (cm)'),
]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Σχεδιάζουμε κάθε ζεύγος σε ξεχωριστό subplot και χρωματίζουμε ανά species
for ax, (xcol, ycol) in zip(axes.flatten(), pairs):
    for cls_idx, cls_name in enumerate(class_names):
        sub = iris_df[iris_df['species'] == cls_name]
        # Use discrete class colormap and norm; c list maps to integer class code
        cvals = np.array([cls_idx] * len(sub))
        ax.scatter(sub[xcol], sub[ycol], label=cls_name, alpha=0.7, c=cvals, cmap=class_cmap, norm=class_norm)
    ax.set_xlabel(xcol)
    ax.set_ylabel(ycol)
    ax.set_title(f"{xcol} vs {ycol}")
    ax.legend(loc='best')

fig.tight_layout()
plt.show()


In [ ]:
# Εξερεύνηση: mean ± std ανά κλάση για κάθε χαρακτηριστικό
# Υπολογίζουμε τον μέσο και την τυπική απόκλιση για κάθε species
means = iris_df.groupby('species')[feature_names].mean()
stds = iris_df.groupby('species')[feature_names].std()

# Σχεδιάζουμε 4 subplots, ένα για κάθε χαρακτηριστικό
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
for ax, feat in zip(axes.flatten(), feature_names):
    # plot mean with errorbars (std) for each species
    x = np.arange(len(means.index))
    ax.bar(x, means[feat], yerr=stds[feat], capsize=6, color=CLASS_COLORS)
    ax.set_xticks(x)
    ax.set_xticklabels(means.index)
    ax.set_title(f"{feat} mean ± std by species")
    ax.set_ylabel(feat)

fig.tight_layout()
plt.show()


In [ ]:
# Train / validation split και κανονικοποίηση με StandardScaler
# Χωρίζουμε το dataset σε train και validation ώστε να αξιολογήσουμε την απόδοση σε κρατημένα δεδομένα
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,  # κρατάμε την αναλογία των κλάσεων σε train & val
    random_state=0,
)

# Δημιουργούμε τον StandardScaler και τον εφαρμόζουμε μόνο στο training set
# ΣΗΜΑΝΤΙΚΟ: ποτέ δεν πρέπει να βασίσουμε τον scaler στο συνολικό dataset (data leakage)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # fit μόνο στο train
X_val_scaled = scaler.transform(X_val)  # transform στο val με το scaler που μάθαμε από το train

# Εκπαίδευση GaussianNB (μονάδα scikit-learn)
# Η GaussianNB υποθέτει κανονικές κατανομές για κάθε χαρακτηριστικό ανά κλάση
gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train)

# Κάνουμε προβλέψεις στο validation set
y_pred = gnb.predict(X_val_scaled)

# Εκτυπώνουμε την αναφορά ταξινόμησης (precision, recall, f1) για κάθε κλάση
print("=== Gaussian Naive Bayes στο Iris dataset ===")
print(
    classification_report(
        y_val,
        y_pred,
        target_names=class_names,
        digits=3,
    )
)


In [ ]:
from IPython.display import display
import pandas as pd

# Απλή και φιλική παρουσίαση των παραμέτρων που έμαθε το GaussianNB
# Θεωρούμε ότι τα gnb, class_names και feature_names έχουν οριστεί σε προηγούμενα cells

feature_names_obj = feature_names if 'feature_names' in globals() else [f'feature_{i}' for i in range(gnb.theta_.shape[1])]
class_names_obj = class_names

# Περίληψη: πλήθος και empirical priors
summary_df = pd.DataFrame({
    'Κλάση': class_names_obj,
    'N_k': gnb.class_count_.astype(int),
    'Empirical prior': gnb.class_prior_
})

# Means and variances per-class (readable with column names)
means_df = pd.DataFrame(gnb.theta_, index=class_names_obj, columns=feature_names_obj)
means_df.index.name = 'Κλάση'
vars_df = pd.DataFrame(gnb.var_, index=class_names_obj, columns=feature_names_obj)
vars_df.index.name = 'Κλάση'


# Display in beginner-friendly format
print('=== GaussianNB learned parameters ===')
print()
display(summary_df)
print('\nΜέσοι ανά κλάση (theta_)')
display(means_df)
print('\nΔιακυμάνσεις ανά κλάση (var_)')
display(vars_df)

# Show shapes and a short note
print('\nShapes:')
print('  theta_.shape =', gnb.theta_.shape)
print('  var_.shape   =', gnb.var_.shape)
print('\nΣημείωση: Οι παραπάνω τιμές αφορούν τυποποιημένα χαρακτηριστικά (StandardScaler).')


## Σύνδεση των παραμέτρων που μάθαμε (learned parameters) με τη θεωρία Gaussian Naive Bayes

Σε αυτό το κελί ερμηνεύουμε τις αριθμητικές τιμές που έμαθε το `GaussianNB` και πώς αυτές μπαίνουν στους θεωρητικούς τύπους για την εκ των υστέρων πιθανότητα (posterior probability) και τον MAP ταξινομητή.

---

### 1. Εμπειρικές εκ των προτέρων πιθανότητες (empirical class priors)

Από την εκπαίδευση πήραμε:

- `class_count_ = [40., 40., 40.]`  
  ⇒ για κάθε κλάση $k$ ισχύει $N_k = 40$ και συνολικά $N = 120$.
- Άρα:
  $$
  \hat{\pi}_k
  = \frac{N_k}{N}
  = \frac{40}{120}
  = \frac{1}{3},
  $$
  που φαίνεται και στο `class_prior_ = [1/3, 1/3, 1/3]`.

Ερμηνεία: το prior $P(y = k)$ είναι **ουδέτερο** (uniform) και δεν προτιμά κάποια κλάση πριν δούμε τα χαρακτηριστικά $x$. Στον τύπο του Bayes αυτό είναι ο όρος:

$$
\log P(y=k) = \log \pi_k.
$$

---

### 2. Μέσοι και διακυμάνσεις ανά κλάση (Gaussian parameters)

Οι πίνακες `theta_` και `var_` δίνουν τους μέσους (means) και τις διακυμάνσεις (variances) των χαρακτηριστικών μετά την τυποποίηση (StandardScaler), για κάθε κλάση:

- **setosa**  
  - μέσοι (means) για τα χαρακτηριστικά:
    - sepal: περίπου [-1.00, 0.82],
    - petal: περίπου [-1.30, -1.26],
  - διακυμάνσεις (variances) στα petal features: περίπου [0.0096, 0.0197] (πολύ μικρές).

- **versicolor**  
  - μέσοι: περίπου [0.05, -0.67, 0.27, 0.17],
  - διακυμάνσεις: περίπου [0.34, 0.53, 0.077, 0.069].

- **virginica**  
  - μέσοι: περίπου [0.95, -0.14, 1.03, 1.09],
  - διακυμάνσεις: περίπου [0.56, 0.54, 0.107, 0.115].

Θεωρητικά, αυτά είναι ακριβώς τα $\hat{\mu}_{k,i}$ και $\hat{\sigma}^2_{k,i}$ στην Κανονική κατανομή (Gaussian pdf):

$$
P(x_i \mid y = k)
= \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2)
= \frac{1}{\sqrt{2\pi\sigma_{k,i}^2}}
  \exp\left(-\frac{(x_i-\mu_{k,i})^2}{2\sigma_{k,i}^2}\right).
$$

Ιδιαίτερα για τη **setosa** βλέπουμε:

- τα petal features έχουν μέσους γύρω στο -1.3,
- με εξαιρετικά μικρές διακυμάνσεις,
- άρα η pdf είναι πολύ «στενή» γύρω από τον μέσο ⇒ δείγματα κοντά σε αυτές τις τιμές έχουν πολύ υψηλή πιθανοφάνεια (likelihood) υπέρ setosa.

---

### 3. Πώς μπαίνουν οι παράμετροι στη log-πιθανοφάνεια (log-likelihood)

Για ένα δείγμα $x = (x_1, \dots, x_d)$ και συγκεκριμένη κλάση $k$, η συνολική (συνεχής) πιθανοφάνεια είναι:

$$
P(x \mid y = k)
= \prod_{i=1}^d \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2).
$$

Σε λογαριθμική κλίμακα (log-scale), που είναι αυτή που χρησιμοποιεί ο `GaussianNB`, έχουμε:

$$
\log P(x \mid y = k)
= \sum_{i=1}^d \log \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2).
$$

Κάθε όρος 
$
\log \mathcal{N}(\cdot)
$
υπολογίζεται από τα αντίστοιχα `theta_[k, i]` και `var_[k, i]`:

$$
\log \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2)
= -\frac{1}{2}
\left[
\log(2\pi\sigma_{k,i}^2)
+
\frac{(x_i - \mu_{k,i})^2}{\sigma_{k,i}^2}
\right].
$$

Άρα οι πίνακες `theta_` και `var_` «μπαίνουν» κατευθείαν μέσα στη log-pdf.

---

### 4. Log-posterior και MAP απόφαση (σύμφωνα με τη θεωρία)

Ο τύπος της εκ των υστέρων πιθανότητας (posterior probability) σε log-scale είναι:

$$
\log P(y=k \mid x)
\propto
\log \pi_k
+
\sum_{i=1}^d \log \mathcal{N}(x_i \mid \mu_{k,i}, \sigma_{k,i}^2).
$$

Αυτό που υπολογίζει εσωτερικά το `GaussianNB` είναι ουσιαστικά:

$$
\text{log\_post}_k
= \log \pi_k
+ \sum_{i=1}^d \log \mathcal{N}(x_i \mid \hat{\mu}_{k,i}, \hat{\sigma}_{k,i}^2),
$$

και ο ταξινομητής επιλέγει την κλάση με μέγιστη log-posterior (MAP estimator):

$$
\hat{y}(x)
= \arg\max_k \text{log\_post}_k.
$$

Η πολύ μικρή διακύμανση στα petal χαρακτηριστικά της `setosa` σημαίνει ότι, όταν ένα δείγμα έχει petal length/width κοντά στις τιμές `[-1.3, -1.26]`, οι log-pdfs αυτών των χαρακτηριστικών για τη setosa γίνονται πολύ μεγαλύτερες από τις αντίστοιχες των άλλων κλάσεων, άρα:

- το $\log P(x \mid y=\text{setosa})$ είναι πολύ μεγαλύτερο,
- και έτσι το $\log P(y=\text{setosa} \mid x)$ κυριαρχεί,
- οπότε η πρόβλεψη είναι σχεδόν βέβαια `setosa`.

Αντίστοιχα:

- τιμές petal κοντά στο ~0.3 ευνοούν `versicolor`,
- τιμές γύρω στο ~1.0 ευνοούν `virginica`.

---

### 5. Ρόλος της τυποποίησης (StandardScaler) και z-scores

Θυμόμαστε ότι:

- εφαρμόσαμε `StandardScaler` **μόνο στο training set** (`fit_transform`),
- και στη συνέχεια χρησιμοποιήσαμε τον ίδιο scaler στο validation set (`transform`),
- αποφεύγοντας data leakage.

Γι’ αυτό οι μέσοι και οι διακυμάνσεις που βλέπουμε (`theta_`, `var_`) αναφέρονται σε **τυποποιημένα χαρακτηριστικά** (z-scores):

$$
z_i = \frac{x_i - \mu_i}{\sigma_i}.
$$

Σε αυτό το z-scale:

- οι «φυσικές» μονάδες (cm) έχουν αφαιρεθεί,
- οι κλάσεις διαχωρίζονται πάνω σε μια κοινή κλίμακα,
- και το GaussianNB μαθαίνει Γκαουσιανές πάνω στο κανονικοποιημένο χώρο, όπως ακριβώς περιγράφει η θεωρία.

Έτσι, τα αποτελέσματα του `classification_report` (υψηλά precision/recall για όλες τις κλάσεις και συνολική accuracy ≈ 0.967) συμφωνούν με την εικόνα που δίνουν οι παράμετροι:  
οι κλάσεις στο Iris είναι καλά διαχωρίσιμες και το Gaussian Naive Bayes, με τα σωστά priors και τις Γκαουσιανές πάνω στα z-scores, τις διακρίνει πολύ αποτελεσματικά.


In [ ]:
# 02 - Σύγκριση κλασσικής μορφής Bayes (γινόμενα pdf) με log-scale υλοποίηση
# Εδώ υπολογίζουμε τους joint scores P(y=k) * Π_i P(x_i | y=k) σε σκέτη μορφή
# και δείχνουμε ότι, μετά από κανονικοποίηση, δίνουν τις ίδιες posterior με την log-based μέθοδο.

import numpy as np

required_vars = ['gnb', 'X_val_scaled', 'class_names']
missing = [name for name in required_vars if name not in globals()]
if missing:
    print("Σφάλμα: Τρέξτε πρώτα τα προηγούμενα cells ώστε να οριστούν " + ", ".join(missing))
else:
    # Gaussian pdf (όχι log) για ένα feature
    def gaussian_pdf(x, mean, var):
        return (1.0 / np.sqrt(2 * np.pi * var)) * np.exp(-0.5 * ((x - mean) ** 2) / var)

    idx = 0
    x_scaled = X_val_scaled[idx]

    raw_joints = []
    per_class_feature_pdfs = []

    for k, cls in enumerate(class_names):
        prior_k = gnb.class_prior_[k]
        feature_pdfs = []
        for i in range(x_scaled.shape[0]):
            mean = gnb.theta_[k, i]
            var = gnb.var_[k, i]
            pdf_i = gaussian_pdf(x_scaled[i], mean, var)
            feature_pdfs.append(pdf_i)
        feature_pdfs = np.array(feature_pdfs)
        per_class_feature_pdfs.append(feature_pdfs)

        product_likelihood = feature_pdfs.prod()         # Π_i P(x_i | y=k)
        raw_joint = prior_k * product_likelihood         # P(y=k) * Π_i P(x_i | y=k)
        raw_joints.append(raw_joint)

    raw_joints = np.array(raw_joints)
    normalized_post_from_raw = raw_joints / raw_joints.sum()

    print(f"Raw unnormalized joint scores P(y=k)*Π_i P(x_i|y=k) για sample idx {idx}:")
    for k, cls in enumerate(class_names):
        print(f"  {cls}: raw_joint={raw_joints[k]:.6e}, normalized posterior ≈ {normalized_post_from_raw[k]:.6f}")

    # Σύγκριση με predict_proba (log-based υλοποίηση του GaussianNB)
    proba = gnb.predict_proba(x_scaled.reshape(1, -1))[0]
    print("\nPosterior από GaussianNB (predict_proba):")
    for k, cls in enumerate(class_names):
        print(f"  {cls}: {proba[k]:.6f}")

    print("\nPer-feature likelihoods P(x_i | y=k):")
    for k, cls in enumerate(class_names):
        print(f"  {cls}: {np.round(per_class_feature_pdfs[k], 6)}")

    # Προαιρετικά: δείχνουμε και τα log-joints για να συνδέσουμε τη μορφή log_prior + Σ log_pdf
    log_joints = []
    for k, cls in enumerate(class_names):
        log_prior_k = np.log(gnb.class_prior_[k])
        log_likelihood_sum = 0.0
        for i in range(x_scaled.shape[0]):
            mean = gnb.theta_[k, i]
            var = gnb.var_[k, i]
            log_likelihood_sum += -0.5 * (np.log(2 * np.pi * var) + ((x_scaled[i] - mean) ** 2) / var)
        log_joints.append(log_prior_k + log_likelihood_sum)
    log_joints = np.array(log_joints)

    log_shift = log_joints - log_joints.max()
    log_normalized = np.exp(log_shift) / np.exp(log_shift).sum()

    print("\nLog-joints (log P(y=k) + Σ_i log P(x_i | y=k)):", np.round(log_joints, 6))
    print("Posterior από log-based normalization          :", np.round(log_normalized, 6))

    print("\nΔίδαγμα: η κλασσική μορφή με γινόμενα pdf και η log-based μορφή είναι μαθηματικά ισοδύναμες,")
    print("αλλά η log-scale υλοποίηση είναι αριθμητικά πολύ πιο σταθερή για πολλά features.")


### Argmax των εκ των υστέρων πιθανοτήτων (posterior) και `predict()` στο GaussianNB

Στα προηγούμενα κελιά είδαμε ότι μπορούμε να υπολογίσουμε για κάθε κλάση $k$ την εκ των υστέρων πιθανότητα
(posterior probability) $P(y=k \mid x)$ είτε στην κλασική μορφή

$$
P(y=k \mid x)
= \frac{P(y=k) \; P(x \mid y=k)}{\sum_j P(y=j) \; P(x \mid y=j)},
$$

είτε σε λογαριθμική κλίμακα (log-scale) ως

$$
\log P(y=k \mid x)
\propto
\log \pi_k
+
\sum_i \log \mathcal{N}(x_i \mid \mu_{k,i}, \sigma^2_{k,i}).
$$

Ο ταξινομητής Gaussian Naive Bayes (GaussianNB) δεν χρειάζεται να επιστρέψει την πλήρη κατανομή — για την πρόβλεψη
της κλάσης αρκεί να βρούμε ποιο $k$ μεγιστοποιεί την posterior. Αυτό γίνεται με τον τελεστή **argmax**:

$$
\hat{y}(x)
=
\arg\max_k \; \log P(y=k \mid x)
=
\arg\max_k \Big( \log \pi_k + \sum_i \log \mathcal{N}(x_i \mid \mu_{k,i}, \sigma^2_{k,i}) \Big).
$$

Στην πράξη:

- η μέθοδος `predict_proba(x)` του `GaussianNB` υπολογίζει τις κανονικοποιημένες εκ των υστέρων πιθανότητες
  (posterior probabilities) $P(y=k \mid x)$,
- η μέθοδος `predict(x)` εφαρμόζει απλώς `argmax` επάνω σε αυτές τις πιθανότητες (ή στα log-posteriors)
  και επιστρέφει την κλάση με τη μεγαλύτερη τιμή.

Για το δείγμα `idx=0` στο Iris dataset, από τον χειροκίνητο υπολογισμό βρήκαμε log-posteriors π.χ.

$$
\text{log\_post} = [-0.935, -37.949, -57.333],
$$

οπότε ο argmax επιλέγει την πρώτη κλάση (`setosa`).  
Η σύγκριση με το `predict_proba` και το `predict` του `GaussianNB` έδειξε ότι:

- οι χειροκίνητα υπολογισμένες posterior $P(y=k \mid x)$ ταιριάζουν με αυτές του μοντέλου,
- το `predict` συμφωνεί με τον argmax επάνω στις log-posteriors.

**Δίδαγμα:** ο GaussianNB υλοποιεί ακριβώς τον θεωρητικό κανόνα του Bayes με Γκαουσιανές pdfs και επιστρέφει ως
πρόβλεψη την κλάση με μέγιστη εκ των υστέρων πιθανότητα (MAP estimator). Οι μικρές διακυμάνσεις (variances) της
`setosa` κάνουν τις αντίστοιχες log-likelihoods πολύ μεγάλες κοντά στους μέσους της, άρα και το posterior για
`setosa` κυριαρχεί σε αυτά τα σημεία του χώρου χαρακτηριστικών.


In [ ]:
# Οπτική απεικόνιση: Gaussian PDFs (scaled) για μήκος πετάλου (petal length) ανά κλάση
# Εδώ σχεδιάζουμε τις per-class likelihood functions P(x_i | y=k) για ένα feature (petal length)
# Αυτές οι συναρτήσεις χρησιμοποιούνται στον κανόνα του Bayes ως πολλαπλασιαστές για κάθε χαρακτηριστικό:
#   P(x | y=k) = Π_i P(x_i | y=k), εδώ βλέπουμε το συστατικό P(petal_length | y=k)

import numpy as np
import matplotlib.pyplot as plt

if 'gnb' not in globals() or 'X_train_scaled' not in globals():
    print('Σφάλμα: Τρέξτε πρώτα το training cell ώστε να οριστούν gnb και X_train_scaled')
else:
    feat_idx = 2  # petal length index
    means = gnb.theta_[:, feat_idx]
    vars_ = gnb.var_[:, feat_idx]

    # Range for x (scaled units)
    x_min, x_max = X_train_scaled[:, feat_idx].min() - 0.5, X_train_scaled[:, feat_idx].max() + 0.5
    xs = np.linspace(x_min, x_max, 400)

    plt.figure(figsize=(8, 4))
    # Plot histogram of training petal length values (scaled)
    plt.hist(X_train_scaled[:, feat_idx], bins=30, density=True, alpha=0.35, label='train petal length (scaled)')

    for k, cls in enumerate(class_names):
        mu = means[k]
        var = vars_[k]
        # pdf of Gaussian: P(x | y=k) = (1 / sqrt(2π var)) * exp(-0.5 * ((x - mu)^2) / var)
        pdf = (1.0 / np.sqrt(2 * np.pi * var)) * np.exp(-0.5 * ((xs - mu) ** 2) / var)
        # plot the per-class likelihood for the feature (one ingredient of Π_i P(x_i | y=k))
        plt.plot(xs, pdf, color=CLASS_COLORS[k], lw=2, label=f'{cls} Gaussian')
        plt.axvline(mu, color=CLASS_COLORS[k], ls='--', alpha=0.7)

    plt.xlabel('petal length (scaled)')
    plt.title('Per-class Gaussian PDFs for petal length (scaled) — these are the per-feature likelihoods P(x_i | y=k)')
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# 05 - Απλό interactive demo: posterior & predict με petal length / width

import importlib.util

if importlib.util.find_spec("ipywidgets") is None:
    print("ipywidgets δεν είναι εγκατεστημένο. Τρέξε: pip install ipywidgets")
else:
    import ipywidgets as widgets
    from ipywidgets import FloatSlider
    from IPython.display import display, clear_output
    import numpy as np
    import matplotlib.pyplot as plt

    # Χρώματα για τις κλάσεις
    if "CLASS_COLORS" in globals():
        colors = CLASS_COLORS
    else:
        colors = ["C0", "C1", "C2"]

    # sliders σε raw cm
    pl_min, pl_max = float(X[:, 2].min()), float(X[:, 2].max())
    pw_min, pw_max = float(X[:, 3].min()), float(X[:, 3].max())

    sl_pl = FloatSlider(
        value=float(X[:, 2].mean()),
        min=pl_min,
        max=pl_max,
        step=0.05,
        description="petal length",
        continuous_update=False,
    )
    sl_pw = FloatSlider(
        value=float(X[:, 3].mean()),
        min=pw_min,
        max=pw_max,
        step=0.05,
        description="petal width",
        continuous_update=False,
    )

    out = widgets.Output()

    def update(petal_length, petal_width):
        with out:
            clear_output(wait=True)

            # δείγμα: sepal στα μέσα τους, petal από sliders
            sepal_mean_0 = X[:, 0].mean()
            sepal_mean_1 = X[:, 1].mean()
            sample_raw = np.array(
                [sepal_mean_0, sepal_mean_1, petal_length, petal_width]
            ).reshape(1, -1)
            sample_scaled = scaler.transform(sample_raw)

            proba = gnb.predict_proba(sample_scaled)[0]
            pred_idx = int(gnb.predict(sample_scaled)[0])

            print("Sample (raw):", np.round(sample_raw.flatten(), 3))
            print("Posterior probs (GaussianNB):")
            for k, cls in enumerate(class_names):
                print(f"  P({cls} | x) = {proba[k]:.3f}")
            print("Predicted class:", pred_idx, "→", class_names[pred_idx])

            # plot
            plt.figure(figsize=(6, 6))
            for k, cls in enumerate(class_names):
                mask = (y == k)
                plt.scatter(
                    X[mask, 2],
                    X[mask, 3],
                    color=colors[k],
                    alpha=0.5,
                    label=cls,
                )
            plt.scatter(
                sample_raw[0, 2],
                sample_raw[0, 3],
                color="k",
                marker="X",
                s=100,
                label="sample",
            )
            plt.xlabel("petal length (cm)")
            plt.ylabel("petal width (cm)")
            plt.title("Interactive sample overlay on petal length vs width")
            plt.legend(title="Class")
            plt.tight_layout()
            plt.show()

    widgets.interactive_output(
        update,
        {"petal_length": sl_pl, "petal_width": sl_pw},
    )

    display(widgets.VBox([sl_pl, sl_pw, out]))


In [ ]:
# Decision boundaries για GaussianNB πάνω στις petal features (μετά την εκπαίδευση)
# Εδώ χρησιμοποιούμε X_train_scaled και y_train που έχουν δημιουργηθεί στο notebook
from matplotlib.colors import ListedColormap

# Επιλέγουμε τις δύο διαστάσεις που θέλουμε να απεικονίσουμε:
# index 2: petal length, index 3: petal width (μετά την τυποποίηση)
X2_train = X_train_scaled[:, [2, 3]]
X2_val = X_val_scaled[:, [2, 3]]

# Εκπαιδεύουμε ένα νέο GaussianNB μόνο με τις δύο αυτές διαστάσεις
clf2 = GaussianNB()
clf2.fit(X2_train, y_train)

# Δημιουργία meshgrid για προβολή decision boundary:
# - x_min..x_max, y_min..y_max προσδιορίζουν την περιοχή που θα καλύψει το mesh
x_min, x_max = X2_train[:,0].min() - 0.5, X2_train[:,0].max() + 0.5
y_min, y_max = X2_train[:,1].min() - 0.5, X2_train[:,1].max() + 0.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))

# Για κάθε σημείο του mesh grid προβλέπουμε την κλάση και μετατρέπουμε το αποτέλεσμα στο σχήμα του grid
Z = clf2.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Χρώματα για το υπόβαθρο (light). Διατηρούμε ένα απαλό colormap για το background
cmap_light = ListedColormap(['#FFEEEE', '#EEFFEE', '#EEEEFF'])
# Use discrete class_cmap for points and class_norm for label mapping

# Σχεδιάζουμε figure και προβάλουμε το decision boundary με contourf
fig, ax = plt.subplots(figsize=(6, 6))
ax.contourf(xx, yy, Z, alpha=0.3, cmap=cmap_light)

# Σχεδιάζουμε τα training σημεία πάνω στο decision boundary για να συγκρίνουμε
# Τα σημεία έχουν χρώματα βάσει της πραγματικής κλάσης (y_train)
ax.scatter(X2_train[:,0], X2_train[:,1], c=y_train, cmap=class_cmap, norm=class_norm, edgecolor='k', s=40)

# Βάζουμε ετικέτες και τίτλο και εμφανίζουμε
ax.set_xlabel('petal length (scaled)')
ax.set_ylabel('petal width (scaled)')
ax.set_title('Decision boundary (GaussianNB) on petal features (train set)')
plt.show()


## Ερμηνεία του ορίου απόφασης (decision boundary)

- Τα χρωματισμένα φόντα δείχνουν τις περιοχές του χώρου χαρακτηριστικών όπου μια κλάση έχει
  τη **μέγιστη εκ των υστέρων πιθανότητα (posterior probability)** $P(y=k \mid x)$.
  Εκεί, ο MAP ταξινομητής (argmax πάνω στο posterior) θα προβλέψει αυτή την κλάση.

- Τα σημεία πάνω στο διάγραμμα είναι τα δείγματα εκπαίδευσης (training samples) και είναι
  χρωματισμένα σύμφωνα με την πραγματική τους κλάση (true class).

Θεωρητική σύνδεση με το Gaussian Naive Bayes (GaussianNB):

- Το GaussianNB υποθέτει ότι κάθε χαρακτηριστικό (feature), για κάθε κλάση (class),
  ακολουθεί μια **Κανονική κατανομή (Gaussian distribution)** με μέσο (mean) και διασπορά (variance)
  $(\mu_{k,i}, \sigma^2_{k,i})$ και κάνει την υπόθεση υπό όρων ανεξαρτησίας (conditional independence)
  μεταξύ των χαρακτηριστικών.

- Η εκ των υστέρων πιθανότητα (posterior probability) υπολογίζεται με τον κανόνα του Bayes
  $P(y \mid x) \propto P(y) \cdot P(x \mid y)$, όπου οι **εκ των προτέρων πιθανότητες
  (class priors)** $P(y)$ και οι Γκαουσιανές pdfs $P(x \mid y)$ καθορίζουν το σχήμα των περιοχών απόφασης.

- Αν όλες οι κλάσεις είχαν ίδιες διακυμάνσεις (ισοτροπική Gaussian), τα όρια απόφασης θα ήταν
  σχεδόν **γραμμικά (linear)**. Με διαφορετικές διακυμάνσεις, εμφανίζονται **καμπύλα / ελλειπτικά**
  σύνορα, όπως στο Iris μεταξύ `versicolor` και `virginica`.

Τυποποίηση (Standardization):

- Οι άξονες του διαγράμματος βρίσκονται σε **τυποποιημένη κλίμακα (standardized z-scores)**, επειδή
  εφαρμόσαμε `StandardScaler` πριν από το GaussianNB. Έτσι οι τιμές δεν είναι σε εκατοστά αλλά σε
  μονάδες τυπικής απόκλισης (standard deviations από τον μέσο).

Συσχέτιση με το Iris dataset:

- Η `Iris setosa` έχει μικρές τιμές στα petal χαρακτηριστικά και μικρή διασπορά, άρα το GaussianNB
  μαθαίνει μια συμπαγή περιοχή με πολύ υψηλό posterior για setosa και ουσιαστικά **κανένα λάθος**.

- Οι `Iris versicolor` και `Iris virginica` έχουν επικαλυπτόμενες κατανομές στα petal features, οπότε
  υπάρχει μια ζώνη όπου τα posteriors είναι κοντινά και τα λάθη (misclassifications) είναι αναμενόμενα.

**Συμπέρασμα:** Το decision boundary είναι μια οπτική απεικόνιση του πού ο ταξινομητής θεωρεί κάθε κλάση πιο
πιθανή, με βάση τους μέσους / διασπορές και τα priors που μάθαμε. Μαζί με το confusion matrix και τις
μετρικές (precision, recall, F1) μας δίνει μια πλήρη εικόνα της συμπεριφοράς του μοντέλου.



In [ ]:
# Αποθήκευση του μοντέλου και του scaler (models/*.joblib)
# Δημιουργούμε φάκελο models κάτω από bayesian_learning/ (ανάλογα με το cwd του notebook)
from pathlib import Path
import joblib

MODELS_DIR = Path('..') / 'bayesian_learning' / 'models'
MODELS_DIR.mkdir(exist_ok=True)

# Αποθηκεύουμε το GaussianNB μοντέλο και τον scaler για μελλοντική χρήση
# joblib.dump δημιουργεί ένα αρχείο .joblib με το εκπαιδευμένο αντικείμενο
joblib.dump(gnb, MODELS_DIR / 'gaussian_nb_iris.joblib')
joblib.dump(scaler, MODELS_DIR / 'gaussian_nb_iris_scaler.joblib')
print(f"Saved model -> {MODELS_DIR / 'gaussian_nb_iris.joblib'}")
print(f"Saved scaler -> {MODELS_DIR / 'gaussian_nb_iris_scaler.joblib'}")


In [ ]:
# Παράδειγμα χρήσης του αποθηκευμένου μοντέλου + scaler
# Φορτώνουμε τα αρχεία που αποθηκεύσαμε παραπάνω και κάνουμε inference
from pathlib import Path
import joblib

MODELS_DIR = Path('..') / 'bayesian_learning' / 'models'

# Φορτώνουμε το joblib αρχείο με το μοντέλο και τον scaler
model_loaded = joblib.load(MODELS_DIR / 'gaussian_nb_iris.joblib')
scaler_loaded = joblib.load(MODELS_DIR / 'gaussian_nb_iris_scaler.joblib')

# Παίρνουμε μερικά δείγματα από το validation set για να συγκρίνουμε τις προβλέψεις
X_sample = X_val[:3]
X_sample_scaled = scaler_loaded.transform(X_sample)

# Προβλέπουμε την κλάση και την πιθανότητα για κάθε δείγμα
pred = model_loaded.predict(X_sample_scaled)
prob = model_loaded.predict_proba(X_sample_scaled)

# Εμφανίζουμε τα αποτελέσματα με κατανόηση: ποιες είναι οι πιθανότητες ανά κλάση
for i, (x, p, pr) in enumerate(zip(X_sample, pred, prob)):
    print("-"*72)
    print(f"Δείγμα #{i}")
    print(f"Χαρακτηριστικά: {x}")
    print(f"Πρόβλεψη κλάσης (index): {p} -> {class_names[p]}")
    for cls_idx, cls_name in enumerate(class_names):
        print(f"  P({cls_name} | x) = {pr[cls_idx]:.3f}")


In [ ]:
# Confusion matrix — απεικόνιση για γρήγορη αξιολόγηση
fig, ax = plt.subplots()
ConfusionMatrixDisplay.from_predictions(
    y_val,
    y_pred,
    display_labels=class_names,
    ax=ax,
    colorbar=False,
)
ax.set_title("Confusion matrix – Gaussian NB (Iris)")
fig.tight_layout()
plt.show()


## Ερμηνεία του confusion matrix

- Το confusion matrix είναι ένα 3×3 πλέγμα όπου οι **σειρές** αντιστοιχούν στις πραγματικές κλάσεις
  (true labels) και οι **στήλες** στις προβλεπόμενες κλάσεις (predicted labels).
- Οι τιμές στη διαγώνιο (diagonal) είναι οι σωστές προβλέψεις (True Positives ανά κλάση).
  Οτιδήποτε εκτός διαγωνίου είναι λάθη (misclassifications).

Για το συγκεκριμένο παράδειγμα του Iris με GaussianNB:

- `setosa`: 10/10 σωστές προβλέψεις (διαγώνιο)  
  → TP = 10, FN = 0, FP = 0 → Precision = 1.000, Recall = 1.000.
- `versicolor`: 10 σωστές προβλέψεις, 1 δείγμα virginica που ταξινομήθηκε ως versicolor  
  → TP = 10, FP = 1, FN = 0 → Precision = 10/(10+1) ≈ 0.909, Recall = 1.000.
- `virginica`: 9 σωστές προβλέψεις, 1 virginica που ταξινομήθηκε ως versicolor  
  → TP = 9, FN = 1, FP = 0 → Precision = 1.000, Recall = 9/(9+1) = 0.900.

Συνολικά:

- Accuracy = (10 + 10 + 9) / 30 = 29/30 ≈ 0.967.
- Το μοναδικό λάθος αφορά τη σύγχυση ανάμεσα σε `versicolor` και `virginica`, κάτι που είδαμε
  και στο decision boundary / scatter plot όπου οι δύο κλάσεις επικαλύπτονται στα petal features.

Πρακτικά συμπεράσματα:

- Η `setosa` είναι πολύ εύκολα διαχωρίσιμη, με μηδενικά λάθη.
- Η επικάλυψη μεταξύ `versicolor` και `virginica` είναι φυσική και εξηγεί τα λίγα λάθη που βλέπουμε.
- Το confusion matrix συμπληρώνει τις μετρικές του `classification_report` (precision, recall, F1),
  δείχνοντας σε ποια ζεύγη κλάσεων εμφανίζεται η σύγχυση.


In [ ]:
# Posterior πιθανότητες για μερικά δείγματα του validation set
# Εδώ χρησιμοποιούμε την sklearn προκειμένου να δείξουμε τις κανονικοποιημένες posterior πιθανότητες
# Σημείωση: gnb.predict_proba(x) επιστρέφει P(y=k | x) για κάθε κλάση

# Επιλέγουμε δείκτες εντός του X_val ώστε να δούμε πραγματικά παραδείγματα
idx_samples = [0, 5, 10]  # δείκτες μέσα στο X_val
X_samples = X_val[idx_samples]
X_samples_scaled = X_val_scaled[idx_samples]
y_true = y_val[idx_samples]

# Προβλέπουμε label και probability για τα δείγματα που επιλέξαμε
# gnb.predict_proba υπολογίζει τις normalized posterior πιθανότητες κατά τον ίδιο τρόπο με τον παραπάνω χειρο-υπολογισμό:
#   post[k] = (P(y=k) * Π_i P(x_i | y=k)) / Σ_k' (P(y=k') * Π_i P(x_i | y=k'))
# Σημείωση: sklearn υπολογίζει αυτόν τον τύπο με log for numerical stability και μετά επιστρέφει κανονικοποιημένες πιθανότητες
y_pred_samples = gnb.predict(X_samples_scaled)
y_proba_samples = gnb.predict_proba(X_samples_scaled)

# Εμφανίζουμε τις πληροφορίες σε αναγνώσιμη μορφή
for i, (x, true_label, pred_label, proba) in enumerate(
    zip(X_samples, y_true, y_pred_samples, y_proba_samples),
):
    print("-" * 72)
    print(f"Δείγμα #{i}")
    print(f"Χαρακτηριστικά (sepal_len, sepal_wid, petal_len, petal_wid): {x}")
    print(f"Πραγματική κλάση: {class_names[true_label]}")
    print(f"Πρόβλεψη       : {class_names[pred_label]}")
    print("Posterior πιθανότητες (P(y=k | x)):")
    # Εδώ εμφανίζουμε τις normalized posterior (predict_proba) που προκύπτουν από τον κανόνα του Bayes
    for cls_idx, cls_name in enumerate(class_names):
        print(f"  P({cls_name} | x) = {proba[cls_idx]:.3f}")
